In [43]:
import os
from dotenv import load_dotenv
from typing import TypedDict

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_groq.chat_models import ChatGroq
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

In [36]:
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.3-70b-versatile")

llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [37]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [38]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
doc_splitter = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splitter,
    collection_name="rag",
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

#### Let's create a RAG chain now

In [39]:
template = """ 
You are an assistant for question-answering task. Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know. Use three sentences maximun and keep the answer concise.
\nQuestion: {question} \nContext: {context} \nAnswer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

In [42]:
question = "tell me about agent memory"
rag_chain.invoke({"context": docs[0], "question": question})

'In the context of LLM-powered autonomous agents, memory refers to the processes used to acquire, store, retain, and retrieve information. There are two types of memory: short-term memory, which utilizes in-context learning and has a limited capacity, and long-term memory, which provides the capability to retain and recall information over extended periods using an external vector store and fast retrieval.'

#### Now lets create grade document class

In [44]:
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    
    binary_score: str = Field(
        description="Documents are relevant to the question 'yes' or 'no'"
    )

In [45]:
structured_llm_grader = llm.with_structured_output(GradeDocuments)

system_msg = """ You are a grader accessing relevance of a retrieved document to a user question. \n
If the document contains keyword(s) or semantic meaning related to question, grade it as relevant. \n
Give binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_msg),
        ("user", "Retrieved document: \n\n{document} \n\n User question: {question}")
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [46]:
question = "tell me about agent memory"
docs = retriever.get_relevant_documents(question)

doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

C:\Users\sowmy\AppData\Local\Temp\ipykernel_14964\3694875819.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


binary_score='yes'


In [47]:
question = "Who built taj Mahal"
docs = retriever.get_relevant_documents(question)

doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


#### Let's create Question Re-Writer

In [48]:
system_msg = """You are a question re-writer that converts an input question to better version that is optimized \n
for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_msg),
        ("user", 
         "Here is the initial question: \n\n {question} \n Formulate an improved quesiton.")
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [49]:
question_rewriter.invoke({"question": question})

'Here\'s an improved version of the question:\n\n"Who was the architect and builder of the Taj Mahal in India?"\n\nThis revised question is more specific and provides more context, making it easier to find accurate and relevant information through a web search. \n\nAlternatively, you could also use:\n\n* "Taj Mahal construction: who was the main architect and builder?"\n* "History of the Taj Mahal: who built it and when?"\n* "Who designed and constructed the Taj Mahal monument in Agra, India?"\n\nThese rephrased questions can help you get more precise and informative results from your search.'

#### Let's create required function

In [50]:
def retrieve(state):
    """ 
    Retrieve documents 
    
    Args: 
        state (dict): The current graph state 
        
    Returns:
    """
    print("---RETRIEVE---")
    
    question = state['question']
    documents = retriever.get_relevant_documents(question)
    
    return {"documents": documents, "question": question}